In [1]:
!pip install matplotlib deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.5 MB/s eta 0:00:00


In [4]:
import random
import numpy as np
import matplotlib.pyplot as plt
from deap import base, creator, tools, algorithms

# Task 1 - Define the number of locations and vehicles
num_locations = 20
locations = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(num_locations)]
depot = (50, 50)
num_vehicles = 3

# Define DEAP classes for fitness and individuals
creator.create("FitnessMin", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(num_locations), num_locations)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define evaluation function
def evalVRP(individual):
    total_distance = 0
    distances = []
    for i in range(num_vehicles):
        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]
        vehicle_distance = sum(np.linalg.norm(np.array(vehicle_route[k+1]) - np.array(vehicle_route[k])) for k in range(len(vehicle_route)-1))
        total_distance += vehicle_distance
        distances.append(vehicle_distance)
    balance_penalty = np.std(distances)
    return total_distance, balance_penalty

toolbox.register("evaluate", evalVRP)

# Alternating Edge Crossover
def cxAlternatingEdge(ind1, ind2):
    """Alternating Edge Crossover (AEX) for permutation-based problems."""
    size = len(ind1)
    child1, child2 = [], []
    available1, available2 = set(ind1), set(ind2)

    current1, current2 = ind1[0], ind2[0]
    for _ in range(size):
        child1.append(current1)
        available1.discard(current1)

        child2.append(current2)
        available2.discard(current2)

        # Alternate parents: pick the next gene from the opposite parent
        if current2 in available1:
            current1 = current2
        else:
            current1 = available1.pop() if available1 else None

        if current1 in available2:
            current2 = current1
        else:
            current2 = available2.pop() if available2 else None

    return creator.Individual(child1), creator.Individual(child2)

toolbox.register("mate", cxAlternatingEdge)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Plot routes
def plot_routes(individual, title="Routes"):
    plt.figure()
    for (x, y) in locations:
        plt.plot(x, y, 'bo')
    plt.plot(depot[0], depot[1], 'rs')
    for i in range(num_vehicles):
        vehicle_route = [depot] + [locations[individual[j]] for j in range(i, len(individual), num_vehicles)] + [depot]
        plt.plot(*zip(*vehicle_route), '-')
    plt.title(title)
    plt.xlabel('X Coordinate')
    plt.ylabel('Y Coordinate')
    plt.show()

# Parameter tuning function
def parameter_tuning(param_grid):
    results = []
    for pop_size in param_grid["pop_size"]:
        for cx_prob in param_grid["cx_prob"]:
            for mut_prob in param_grid["mut_prob"]:
                for tournsize in param_grid["tournsize"]:
                    random.seed(42)
                    toolbox.register("select", tools.selTournament, tournsize=tournsize)
                    pop = toolbox.population(n=pop_size)
                    hof = tools.HallOfFame(1)
                    stats = tools.Statistics(lambda ind: ind.fitness.values)
                    stats.register("avg", np.mean)
                    stats.register("min", np.min)

                    algorithms.eaSimple(pop, toolbox, cx_prob, mut_prob, ngen=100, stats=stats, halloffame=hof, verbose=False)

                    best_fitness = hof[0].fitness.values
                    diversity = calculate_diversity(pop)
                    results.append({
                        "pop_size": pop_size,
                        "cx_prob": cx_prob,
                        "mut_prob": mut_prob,
                        "tournsize": tournsize,
                        "best_fitness": best_fitness,
                        "diversity": diversity,
                        "hof": hof[0]
                    })
    return results

# Calculate diversity
def calculate_diversity(population):
    diversity = 0
    n = len(population)
    for i in range(n):
        for j in range(i + 1, n):
            diversity += np.sum(np.array(population[i]) != np.array(population[j]))
    return diversity / (n * (n - 1) / 2)

# Main function
def main():
    param_grid = {
        "pop_size": [100, 200, 300],
        "cx_prob": [0.6, 0.7, 0.8],
        "mut_prob": [0.1, 0.2, 0.3],
        "tournsize": [3, 5]
    }

    results = parameter_tuning(param_grid)

    for result in results:
        print(f"Population: {result['pop_size']}, Crossover: {result['cx_prob']}, Mutation: {result['mut_prob']}, "
              f"Tournament Size: {result['tournsize']}, Best Fitness: {result['best_fitness']}, "
              f"Diversity: {result['diversity']}")

        # Optionally, plot the best route
        # plot_routes(result['hof'], title=f"Best Route (Pop: {result['pop_size']}, Cx: {result['cx_prob']})")

if __name__ == "__main__":
    main()


Population: 100, Crossover: 0.6, Mutation: 0.1, Tournament Size: 3, Best Fitness: (836.9249271608556, 49.783408854981836), Diversity: 10.30929292929293
Population: 100, Crossover: 0.6, Mutation: 0.1, Tournament Size: 5, Best Fitness: (704.6445557512498, 65.02114513112123), Diversity: 8.756565656565657
Population: 100, Crossover: 0.6, Mutation: 0.2, Tournament Size: 3, Best Fitness: (745.2175652621484, 51.23859411266992), Diversity: 10.153939393939394
Population: 100, Crossover: 0.6, Mutation: 0.2, Tournament Size: 5, Best Fitness: (639.928745683379, 35.56847229792306), Diversity: 9.64080808080808
Population: 100, Crossover: 0.6, Mutation: 0.3, Tournament Size: 3, Best Fitness: (775.226422673454, 68.93294573407917), Diversity: 12.83131313131313
Population: 100, Crossover: 0.6, Mutation: 0.3, Tournament Size: 5, Best Fitness: (584.8855227010697, 39.226172727299115), Diversity: 10.555757575757577
Population: 100, Crossover: 0.7, Mutation: 0.1, Tournament Size: 3, Best Fitness: (836.924927